Copyright 2020 Konstantin Yakovlev, Matthias Anderer

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [2]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)


In [3]:
LOSS_MULTIPLIER = 0.99 # Set multiplier according to desired under-/overshooting

In [4]:
# define custom loss function
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * LOSS_MULTIPLIER)
    hess = np.where(residual < 0, 2, 2 * LOSS_MULTIPLIER)
    return grad, hess

# define custom evaluation metric
def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * LOSS_MULTIPLIER) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [5]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]
    
    ############
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


In [6]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'tweedie',
        'tweedie_variance_power': 1.1,
        'metric':'rmse',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.2,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        'lambda_l1': 0.5,
        'lambda_l2': 0.5
}



In [7]:
########################### Vars
#################################################################################
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913+28            # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
USE_AUX     = False               # Use or not pretrained models

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]

#PATHS for Features
ORIGINAL = 'C://Users//nkyam//Desktop//m5_forecast//'
BASE     = 'C://Users//nkyam//Desktop//m5_forecast//grid_part_1.pkl'
PRICE    = 'C://Users//nkyam//Desktop//m5_forecast//grid_part_2.pkl'
CALENDAR = 'C://Users//nkyam//Desktop//m5_forecast//grid_part_3.pkl'

#STORES ids
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())
STORES_IDS 


['CA_1',
 'CA_2',
 'CA_3',
 'CA_4',
 'TX_1',
 'TX_2',
 'TX_3',
 'WI_1',
 'WI_2',
 'WI_3']

In [8]:
########################### Train Models
#################################################################################
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validatio set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          num_boost_round = 3600, 
                          early_stopping_rounds = 50, 
                          valid_sets = [train_data, valid_data],
                          verbose_eval = 100,
                          fobj = custom_asymmetric_train

                          )
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
[LightGBM] [Info] Saving data to binary file train_data.bin
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.134519 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.57135	valid_1's rmse: 2.22931
[200]	training's rmse: 2.47298	valid_1's rmse: 2.1568
[300]	training's rmse: 2.41283	valid_1's rmse: 2.11583
[400]	training's rmse: 2.37198	valid_1's rmse: 2.07618
[500]	training's rmse: 2.33781	valid_1's rmse: 2.06178
[600]	training's rmse: 2.3077	valid_1's rmse: 2.0517
[700]	trai

'rm' is not recognized as an internal or external command,
operable program or batch file.


Train CA_2
[LightGBM] [Warning] File train_data.bin exists, cannot save binary to it
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.129244 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[10]	training's rmse: 2.93869	valid_1's rmse: 2.44416
Train CA_3


'rm' is not recognized as an internal or external command,
operable program or batch file.


[LightGBM] [Warning] File train_data.bin exists, cannot save binary to it
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130817 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.57135	valid_1's rmse: 3.43899
[200]	training's rmse: 2.47298	valid_1's rmse: 3.43128
Early stopping, best iteration is:
[229]	training's rmse: 2.44891	valid_1's rmse: 3.40975
Train CA_4


...

'rm' is not recognized as an internal or external command,
operable program or batch file.


[LightGBM] [Warning] File train_data.bin exists, cannot save binary to it
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.111248 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[7]	training's rmse: 3.05678	valid_1's rmse: 4.5451
Train WI_3


'rm' is not recognized as an internal or external command,
operable program or batch file.


[LightGBM] [Warning] File train_data.bin exists, cannot save binary to it
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.113239 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.57135	valid_1's rmse: 2.87899
Early stopping, best iteration is:
[94]	training's rmse: 2.57864	valid_1's rmse: 2.87581


'rm' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
########################### Predict
#################################################################################

# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  0.03 min round |  0.03 min total |  36343.21 day sales |
Predict | Day: 2
##########  0.03 min round |  0.07 min total |  33751.66 day sales |
Predict | Day: 3
##########  0.03 min round |  0.10 min total |  33695.24 day sales |
Predict | Day: 4
##########  0.03 min round |  0.14 min total |  33614.33 day sales |
Predict | Day: 5
##########  0.03 min round |  0.17 min total |  37560.45 day sales |
Predict | Day: 6
##########  0.03 min round |  0.20 min total |  45945.89 day sales |
Predict | Day: 7
##########  0.03 min round |  0.24 min total |  46580.85 day sales |
Predict | Day: 8
##########  0.03 min round |  0.27 min total |  38980.65 day sales |
Predict | Day: 9
##########  0.03 min round |  0.31 min total |  34390.58 day sales |
Predict | Day: 10
##########  0.03 min round |  0.34 min total |  36645.56 day sales |
Predict | Day: 11
##########  0.03 min round |  0.37 min total |  35391.07 day sales |
Predict | Day: 12
##########  0.03 min round |  0.41

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.697756,0.621181,0.617940,0.627945,0.736173,0.858946,0.835927,0.746057,0.704763,...,0.743521,0.872772,0.850420,0.683640,0.620496,0.619237,0.625387,0.734829,0.887465,0.767701
1,HOBBIES_1_002_CA_1_evaluation,0.309601,0.233026,0.229786,0.239790,0.325206,0.468542,0.449761,0.356225,0.319403,...,0.337256,0.479842,0.464401,0.295632,0.232488,0.231229,0.237379,0.324962,0.451904,0.341570
2,HOBBIES_1_003_CA_1_evaluation,0.386408,0.309833,0.306593,0.316597,0.402013,0.545348,0.526568,0.440747,0.396209,...,0.414063,0.556649,0.541208,0.372439,0.309295,0.308036,0.314185,0.401769,0.528711,0.418377
3,HOBBIES_1_004_CA_1_evaluation,1.619282,1.465118,1.461877,1.464726,2.084660,2.993645,3.460564,1.914311,1.537374,...,1.999493,3.000730,3.149320,1.600042,1.450537,1.449278,1.455428,2.077529,2.970121,2.930912
4,HOBBIES_1_005_CA_1_evaluation,0.973867,0.893747,0.890507,0.900511,1.044945,1.343865,1.323175,1.022639,0.983986,...,1.022508,1.413680,1.396330,1.014868,0.951724,0.950465,0.956615,1.102261,1.399429,1.286832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.938749,0.809738,0.809738,0.809738,0.923374,1.303247,1.307568,1.054843,0.809738,...,0.999479,1.332587,1.336006,0.938749,0.827629,0.827629,0.827629,0.941264,1.321138,1.318777
30486,FOODS_3_824_WI_3_evaluation,0.796970,0.640877,0.640877,0.640877,0.664574,1.044448,1.048769,0.941245,0.640877,...,0.767762,1.100870,1.104289,0.796970,0.685850,0.685850,0.685850,0.709548,1.089421,1.087060
30487,FOODS_3_825_WI_3_evaluation,1.035912,0.879819,0.879819,0.879819,1.022221,1.402095,1.406416,1.124923,0.879819,...,1.142478,1.475585,1.479005,1.052981,0.941861,0.941861,0.941861,1.084263,1.464137,1.461776
30488,FOODS_3_826_WI_3_evaluation,1.355020,1.217239,1.217239,1.217239,1.509090,1.872274,1.876595,1.462343,1.217239,...,1.593967,1.910385,1.913804,1.355020,1.243901,1.243901,1.243901,1.535752,1.898936,1.867702


In [10]:
########################### Export
#################################################################################
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission_v'+str(VER)+'.csv', index=False)